## Data

In [1]:
import os
import sys

# a crutch to add parent dir to sys.path to be able to import dataset module
sys.path.append(os.getcwd()[: -len("experiments") - 1])

In [2]:
from pathlib import Path
from dataset.datasets_schema import Datasets

datasets_path = Path("../dataset/datasets.yaml")
datasets: Datasets = Datasets.from_yaml(datasets_path)
moodle_dataset = datasets["moodle"]
entries = moodle_dataset.document_storage.entries
print(len(entries))

821


In [3]:
import numpy as np

np.random.seed(42)

keys = np.random.choice(list(entries.keys()), 100)
keys

array(['files/module-109689.pdf', 'files/module-85174.pdf',
       'files/module-83177.pdf', 'files/module-109753.pdf',
       'files/module-109314.pdf', 'files/module-92944.pdf',
       'files/module-107797.pdf', 'files/module-91849.pdf',
       'files/module-109918.pdf', 'files/module-89453.pdf',
       'files/module-82581.pdf', 'files/module-83870.pdf',
       'files/module-87970.pdf', 'files/module-109542.pdf',
       'files/module-84456.pdf', 'files/module-109667.pdf',
       'files/module-92420.pdf', 'files/module-110086.pdf',
       'files/module-92408.pdf', 'files/module-83571.pdf',
       'files/module-93941.pptx', 'files/module-84038.pdf',
       'files/module-89513.pdf', 'files/module-84850.pdf',
       'files/module-94473.pdf', 'files/module-84599.pdf',
       'files/module-79202.pdf', 'files/module-83283.pdf',
       'files/module-78546.pdf', 'files/module-87975.pdf',
       'files/module-83659.pdf', 'files/module-108075.pdf',
       'files/module-82987.pdf', 'files/module

## MinIO -> (pdfs)

minio = "^7.2.7"
sentence-transformers = "^3.0.1"
faiss-cpu = "^1.8.0.post1"

Install minio image and run its created container  

In [ ]:
!docker pull -q quay.io/minio/minio

In [ ]:
!docker run -d -p 9000:9000 -p 9001:9001 --name minio -e "MINIO_ROOT_USER=minioadmin" -e "MINIO_ROOT_PASSWORD=password" -v D:\data:/data quay.io/minio/minio server /data --console-address ":9001"

Connenct to the minio

In [4]:
from minio import Minio, S3Error

# create client with access key and secret key with specific region.
client = Minio(
    endpoint="127.0.0.1:9000",  # 9090
    access_key="minioadmin",
    secret_key="password",
    secure=False,
)

In [ ]:
"""
might be a solution to search api problem
Create client with custom HTTP client using proxy server.
# import urllib3
# client = Minio(
#     "SERVER:PORT",
#     access_key="ACCESS_KEY",
#     secret_key="SECRET_KEY",
#     secure=True,
#     http_client=urllib3.ProxyManager(
#         "https://PROXYSERVER:PROXYPORT/",
#         timeout=urllib3.Timeout.DEFAULT_TIMEOUT,
#         cert_reqs="CERT_REQUIRED",
#         retries=urllib3.Retry(
#             total=5,
#             backoff_factor=0.2,
#             status_forcelist=[500, 502, 503, 504],
#         ),
#     ),
# )
"""

In [18]:
BUCKET_NAME = "moodle"

if not client.bucket_exists(BUCKET_NAME):
    client.make_bucket(BUCKET_NAME)

client.list_buckets()

[Bucket('moodle')]

In [19]:
for key in keys:
    try:
        file_path = str(entries[key].path)
        print(file_path)
        with open(file_path, "rb") as file:
            file_stat = os.stat(file_path)

            client.put_object(
                bucket_name=BUCKET_NAME,
                object_name=key,
                data=file,
                length=file_stat.st_size,
                content_type="application/pdf",
            )
    except S3Error as e:
        print("error occurred.", e)

..\data\moodle\files\module-109689.pdf
..\data\moodle\files\module-85174.pdf
..\data\moodle\files\module-83177.pdf
..\data\moodle\files\module-109753.pdf
..\data\moodle\files\module-109314.pdf
..\data\moodle\files\module-92944.pdf
..\data\moodle\files\module-107797.pdf
..\data\moodle\files\module-91849.pdf
..\data\moodle\files\module-109918.pdf
..\data\moodle\files\module-89453.pdf


In [24]:
objects = client.list_objects(BUCKET_NAME, recursive=True)
for obj in objects:
    print(obj)

In [ ]:
os.remove(file_path)

In [26]:
# get file from minio
try:
    client.fget_object(BUCKET_NAME, keys[0], keys[0])
except S3Error as e:
    print(f"Error occured: {e.message}")

In [27]:
os.remove(keys[0])  # delete in order not store

## pymupdf4llm -> (md)

In [5]:
from tqdm.notebook import tqdm
from pymupdf4llm import process_document, join_chunks

# entries = [entry for entry in entries if entry.endswith(".pdf")]
processed_documents = [process_document(entries[key].path) for key in keys if key.endswith(".pdf")]

In [6]:
texts = []
for result in tqdm(processed_documents, total=len(keys)):
    texts.append(join_chunks(result))

  0%|          | 0/100 [00:00<?, ?it/s]

In [7]:
texts[0]

'<h2>Computer Networks</h2>\n<h2>Lecture Week 11 (Part I)</h2>\n\n<h1><b>TCP Congestion Control:</b></h1>\n<h1><b>The Basic Principles</b></h1>\n\nArtem Burmyakov\n\nApril 05, 2024\n\n-----\n\nRecap: Router Architecture\n\nForwarding Table\n\n-----\n\nRecap: Router Architecture\n\nForwarding Table\n\n<b>Routers and other network core devices may cause various communication problems</b>\n\n-----\n\nRecap: Router Architecture\n\nForwarding Table\n\n<b>Routers and other network core devices may cause various communication problems;</b>\n<b>A packet loss is one of them, due to a buffer overflow at a router</b>\n\n-----\n\nReference: TCP/IP Reference Model\n\n-----\n\nReference: TCP/IP Reference Model <b>Question:</b>\n<b>Which layer(s) is/are responsible for</b>\n<b>detecting and fixing network-related problems?</b>\n\n-----\n\nReference: TCP/IP Reference Model <b>Question:</b>\n<b>Which layer(s) is/are responsible for</b>\n<b>detecting and fixing network-related problems?</b>\n\n<b>Exampl

## Preprocess -> (str)

In [8]:
import re
import unicodedata
from bs4 import BeautifulSoup

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer  # , TreebankWordTokenizer

nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)

True

In [9]:
def normalize(text: str) -> str:
    return unicodedata.normalize("NFKC", text)


def to_lowercase(text: str) -> str:
    return text.lower()


def remove_punctuation(text: str) -> str:
    return re.sub(r"[^\w\s]", "", text)


def remove_html(text: str) -> str:
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text(separator=" ")


def remove_markdown(text: str) -> str:
    # TODO: ...
    return text


def remove_urls(text: str) -> str:
    url_pattern = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
    return re.sub(url_pattern, "", text)


def remove_stop_words(words: list[str]) -> list[str]:
    stop_words = set(stopwords.words("english")) | set(stopwords.words("russian"))
    filtered_words = [word for word in words if word.casefold() not in stop_words]
    return filtered_words

In [10]:
def test_preprocessing():
    assert normalize("𝑃𝑜𝑠𝑖𝑡𝑖𝑣𝑒𝑠") == "Positives"
    assert to_lowercase("Omg Hiiiii") == "omg hiiiii"
    assert remove_html("<h2><b>Deep Learning for Search</b></h2>") == "Deep Learning for Search"
    assert remove_punctuation("Hello, quite! A lot? :_+") == "Hello quite A lot _"
    # assert remove_markdown("") == ""
    assert remove_urls("Here is the link https://hotel.innopolis.university/dokumenty/ !") == "Here is the link  !"
    # assert remove_stop_words([]) == []


test_preprocessing()

In [11]:
def preprocess(text: str, tokenizer) -> str:
    text = normalize(text)  # . normalize
    text = to_lowercase(text)  # to lower case (think of Names etc)
    text = remove_html(text)  # remove tags
    text = remove_punctuation(text)  # remove punctuation
    # text = remove_markdown(text)
    text = remove_urls(text)  # remove urls

    # TOKENIZE #
    words = tokenizer.tokenize(text)
    words = remove_stop_words(words)

    return " ".join(words)

In [12]:
texts = [preprocess(text, WordPunctTokenizer()) for text in texts]

In [13]:
texts[0]

'computer networks lecture week 11 part tcp congestion control basic principles artem burmyakov april 05 2024 recap router architecture forwarding table recap router architecture forwarding table routers network core devices may cause various communication problems recap router architecture forwarding table routers network core devices may cause various communication problems packet loss one due buffer overflow router reference tcpip reference model reference tcpip reference model question layers isare responsible detecting fixing networkrelated problems reference tcpip reference model question layers isare responsible detecting fixing networkrelated problems examples possible problems packet loss packet corruption slow communication speed reference tcpip reference model question layers isare responsible providing reliable network communication reference tcpip reference model question layers isare responsible providing reliable network communication reference tcpip reference model ques

## Junking -> (junks)

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [15]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=150,
    chunk_overlap=20,
    length_function=lambda x: len(x.split()),  # 150 words, not characters
    add_start_index=True,
)

In [16]:
# class ChunksInfo(BaseModel):
chunks: list[str] = []
indexes: list[int] = []
mappings: dict[int, str] = {}  #  mapping chunk index to document

In [17]:
for i, text in enumerate(texts):
    text_chunks = text_splitter.split_text(text)
    amount = len(text_chunks)
    chunks.extend(text_chunks)
    indexes.extend([i] * amount)
    mappings.update({i: keys[i]})

In [18]:
len(chunks), len(indexes)

(601, 601)

## Embeddings -> (vectors)

In [19]:
from sentence_transformers import SentenceTransformer

# MODEL_NAME = 'all-mpnet-base-v2'  # SOTA
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(MODEL_NAME)

c:\Users\PodYapolsky\AppData\Local\pypoetry\Cache\virtualenvs\search-experiments-AGuved54-py3.11\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [20]:
embeddings = model.encode(chunks)

c:\Users\PodYapolsky\AppData\Local\pypoetry\Cache\virtualenvs\search-experiments-AGuved54-py3.11\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [21]:
embeddings.shape

(601, 384)

## FAISS

Check for normalization of embeddings

In [31]:
import numpy as np

i = 0
for embedding in embeddings:
    if abs(np.linalg.norm(embedding) - 1) > 1e-5:
        i += 1

print(i)

0


Since vectors are normalized, IP (Inner Product) will work as cosine

https://github.com/facebookresearch/faiss/wiki/Faiss-indexes

https://github.com/facebookresearch/faiss/issues/95


In [32]:
import faiss

n, d = embeddings.shape
index = faiss.IndexFlatIP(d)

In [33]:
index.is_trained

True

In [34]:
index.add(embeddings)

Query

In [35]:
query_embedding = model.encode(["General form of quadrics"])

In [36]:
D, I = index.search(query_embedding, 10)  # noqa: E741

First chunk is the closest as we expected

In [37]:
print(D)
print(I)

[[0.663553   0.5191452  0.49850857 0.48450506 0.46675572 0.45633268
  0.44380134 0.43932167 0.4006983  0.38222015]]
[[ 93 573  96  97  98  95  94  99 411 226]]


In [39]:
print(chunks[I[0][0]])
print(chunks[I[0][1]])
print(chunks[I[0][2]])

quadrics teaching assistant eugene marchuk december 1 2022 quadrics general form know seconddegree polynomial equation two variables conic written x 2 b xy c 2 x e f 0 quadric given x 2 b 2 c z 2 xy e xz f yz g x h z j 0 clearly first six coefficients vanish equation linear describes plane space general case equation describes classic quadrics ellipsoids paraboloids hyperboloids 2 24 quadrics euclidean plane e 2 3 24 quadrics euclidean space e 3 4 24 quadrics euclidean space e 3 5 24 quadrics euclidean space e 3 remark definitions given conical central parabolic quadrics depend type quadratic form polynomial equation surface matrix notation see b odehnal h stachel g glaeser universe quadrics springer berlin heidelberg 2020 6 24 quadrics example 1 lab 14 task 1 find equation cone vertex v 1 1 1 passes curve x 2 2 4 z 2
1 x 2 1 0 1 0 x 1 x 2 1 0 finally 6 ε x 5 ε qed 0 ps im sure solution ε correct another example solution possible right track 11 28 problem 2 pivots rank eigenvalues dete

In [44]:
print(indexes[I[0][0]])
print(indexes[I[0][1]])
print(indexes[I[0][2]])

1
91
1


Collect file paths to return

In [40]:
file_paths = []
added_paths = set()

for i in I[0]:
    doc_index = indexes[i]

    file_path = mappings[doc_index]
    if file_path not in added_paths:
        added_paths.add(file_path)
        file_paths.append(file_path)

file_paths

['files/module-85174.pdf',
 'files/module-87984.pdf',
 'files/module-84395.pdf',
 'files/module-83283.pdf']

In [43]:
for key in file_paths:
    print(entries[key].path)

..\data\moodle\files\module-85174.pdf
..\data\moodle\files\module-87984.pdf
..\data\moodle\files\module-84395.pdf
..\data\moodle\files\module-83283.pdf


In [ ]:
for file_path in file_paths:
    client.fget_object(BUCKET_NAME, file_path, file_path)

Try apply Reranker

https://github.com/FlagOpen/FlagEmbedding/tree/master/FlagEmbedding/llm_reranker
